# Landcover Statistics Processing Script

**Author:** Gabriel Ortega

## Libraries

In [1]:
import ee
ee.Authenticate()
ee.Initialize(project='gortega-research')

from datetime import datetime

## 1. Settings

In [2]:
# Polygons collection
table = ee.FeatureCollection("projects/gortega-research/assets/wdpa_workdata")  # Explicitly define WDPA polygons
# Unique polygons identifier
uniqueID = 'wdpaid'

# landcover Dataset
landcoverDataset = ee.ImageCollection("COPERNICUS/Landcover/100m/Proba-V-C3/Global")  # Use the Copernicus land cover
landcoverBand = 'discrete_classification'  # Band name for land cover
MaskBand = 'discrete_classification'  # Band name for mask

# Full bounding box
xmin, ymin, xmax, ymax = -180, 90, 180, -90

# Range
rangestart = 501
rangeend = 1025

# List of possible landcover values (based on Copernicus Landcover documentation)
landcover_values = ['20', '30', '40', '50', '60', '70', '80', '90', '100','111', '112', '113', '114', '115', '116', '121', '122', '123', '124', '125', '126', '200']

# Get the current date
current_date = datetime.now().strftime('%Y-%m-%d')
export_folder = f'landcover_wdpa_{current_date}'

## 2. Function(s)

In [3]:
def processData():
  def calculateStats(feature):
    combinedGeometry = feature.geometry()

    # Use groupField=0 (landcover band) to count 'counter' band
    landcoverStats = maskedlandcover.reduceRegion(
      reducer=ee.Reducer.count().group(
        groupField=0,  # Group by landcover band (index 0)
        groupName='lc_class'
      ),
      geometry=combinedGeometry,
      scale=100,
      maxPixels=1e13,
      bestEffort=True,
      tileScale=16
    )

    # Get groups list or empty list if key missing
    groupList = ee.List(landcoverStats.get('groups', ee.List([])))

    # Initialize counts with fallback values
    landcover_counts = ee.Dictionary.fromLists(
        ee.List(landcover_values).map(lambda x: ee.String(x)),
        ee.List.repeat(0, len(landcover_values))
    )

    def updateCounts(item, currentDict):
        item = ee.Dictionary(item)
        group = ee.Number(item.get('lc_class', -9999))  # Correct key from groupName
        count = ee.Number(item.get('count', 0))
        return ee.Dictionary(currentDict).set(group.format('%d'), count)  # Ensure string key

    updated_landcover_counts = ee.Dictionary(groupList.iterate(updateCounts, landcover_counts))

    # Get the total number of pixels in the combined polygon.
    totalPixels = maskedlandcover.reduceRegion(
      reducer=ee.Reducer.count(),
      geometry=combinedGeometry,
      scale=100,
      maxPixels=1e13,
      bestEffort=True,
      tileScale=16
    ).get(landcoverBand)

    # Prepare results dictionary
    # Use ee.Dictionary.combine to merge the landcover counts with total pixels
    results = updated_landcover_counts.combine(ee.Dictionary({'total_pixels': totalPixels}))

    return ee.Feature(None, results).set(uniqueID, feature.get(uniqueID))

  features = simplifiedPols.map(calculateStats)
  return ee.FeatureCollection(features)

## 3. Execution

In [4]:

# Loop over whichBox
for whichBox in range(rangestart, rangeend):
  # Initialize bbox variable
  if whichBox == 0:
    # Full bounding box
    bbox = ee.Geometry.BBox(xmin, ymin, xmax, ymax)
  else:
    # Grid configuration
    xSegments, ySegments = 32, 32

    # Calculate step sizes
    xStep = (xmax - xmin) / xSegments  # 45° per column
    yStep = (ymax - ymin) / ySegments  # -22.5° per row

    # Convert whichBox to grid indices
    adjustedIndex = whichBox - 1
    row = adjustedIndex // xSegments  # 0-3 (bottom to top)
    col = adjustedIndex % xSegments  # 0-3 (left to right)

    # Reverse row order (original grid is top to bottom)
    yIndex = (ySegments - 1) - row

    # Calculate coordinates
    xStart = xmin + col * xStep
    xEnd = xStart + xStep
    yStart = ymin + yIndex * yStep
    yEnd = yStart + yStep

    bbox = ee.Geometry.BBox(xStart, yStart, xEnd, yEnd)

  # Name of output file
  fileName = f'landcover_stats_WDPA_{xStart}_{yStart}_{xEnd}_{yEnd}_bbox{whichBox}'

  # Filter polygons dataset as required
  filteredPols = table.filterBounds(bbox)

  # Check if there are any features in the filtered collection before proceeding
  if filteredPols.size().getInfo() > 0:

    # Apply simplify to the geometry of each feature using map
    simplifiedPols = filteredPols.map(lambda feature: feature.simplify(maxError=10))

    # Get landcover image with counter band
    filteredLC = landcoverDataset.filterBounds(simplifiedPols.geometry())
    landcover = filteredLC.select(landcoverBand).mosaic()
    maskedlandcover = landcover.updateMask(landcover.neq(0))

    # Create pixel counter band
    maskedlandcover = maskedlandcover.addBands(ee.Image.constant(1).rename('counter'))

    allFeatures = processData()

    ############################################################################################################################
    # 4. EXPORT RESULTS

    task = ee.batch.Export.table.toDrive(
      collection=allFeatures,
      description=fileName,
      fileFormat='CSV',
      folder=export_folder
    )
    task.start()